# Notebook for training the YOLOv8 model

# Train the model

In [1]:
from ultralytics import YOLO

# Load a model
#model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="config.yaml", epochs=10)  # train the model
metrics = model.val()  # evaluate model performance on the validation set

#results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
#path = model.export(format="onnx")  # export the model to ONNX format

New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=config.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_con

train: Scanning /home/jetracer/Documents/3d_mai/data/train_threads_b/object_train_val/train/labels... 1500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1500/1500 [00:00<00:00, 3920.33it/s]

train: New cache created: /home/jetracer/Documents/3d_mai/data/train_threads_b/object_train_val/train/labels.cache



val: Scanning /home/jetracer/Documents/3d_mai/data/train_threads_b/object_train_val/train/labels.cache... 1500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1500/1500 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


10 epochs completed in 1.412 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [01:21<00:00,  1.74s/it]


                   all       1500       2090      0.967       0.93       0.98      0.917
                   nut       1500        573      0.962      0.829      0.965      0.808
           screw-small       1500        617      0.946      0.912      0.964       0.87
                washer       1500        300      0.998          1      0.995      0.978
           screw-large       1500        300      0.945      0.953      0.987      0.973
              assembly       1500        300      0.985      0.957       0.99      0.956
Speed: 1.2ms preprocess, 41.9ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/train
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/jetracer/Documents/3d_mai/data/train_threads_b/object_train_val/train/labels.cache... 1500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1500/1500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [01:19<00:00,  1.19it/s]


                   all       1500       2090      0.967       0.93       0.98      0.917
                   nut       1500        573      0.962      0.829      0.965      0.808
           screw-small       1500        617      0.946      0.912      0.964       0.87
                washer       1500        300      0.998          1      0.995      0.978
           screw-large       1500        300      0.945      0.953      0.987      0.973
              assembly       1500        300      0.985      0.957       0.99      0.956
Speed: 1.2ms preprocess, 39.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/train2


# Inference

In [1]:
from pathlib import Path
from ultralytics import YOLO

def yolo_inference(source, weights):
    source_path = Path(source).absolute()
    weights_path = Path(weights).absolute()

    # Create YOLOv8 object
    yolo = YOLO(weights_path) #, imgsz=640)  # Adjust imgsz as needed

    yolo.val()
    # Run inference
    #results = yolo(source_path)

    # Display or process results as needed
    #print(results)

if __name__ == "__main__":
    source_path = "/home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/raw"
    weights_path = "/home/jetracer/Documents/3d_mai/notebooks/yolo_model/runs/threads-assembly/weights/best.pt"

    yolo_inference(source=source_path, weights=weights_path)


Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/test/labels... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<00:00, 559.32it/s]

val: New cache created: /home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.80it/s]


                   all          9         18          1      0.302      0.523      0.278
                   nut          9          9          1          0      0.138     0.0482
           screw-small          9          9          1      0.605      0.908      0.508
Speed: 0.9ms preprocess, 55.3ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val2
